# Image Processing for CZI Files

## Splitting the combined .czi file into separate channels

### List of Tasks

1. Read CZI file from a given folder.
2. Read metadata for each file and based on channel number split into separate channels.
3. Save each channel as a separate tif/tiff file with correct channel name without permission errors.

In [1]:
import glob
import os
import re
import shutil
import time
import cv2 as cv
import imageio 
import nrrd
import numpy as np
import SimpleITK as sitk
import tifffile as tiff
from aicspylibczi import CziFile
from scipy import ndimage
from skimage import exposure, io
import matplotlib.pyplot as plt
from skimage import data
from aicsimageio import AICSImage

### Enter the exact/full path of the folder where images are located.

In [2]:
c_path = r"C:\Users\keshavgubbi\Desktop\LinesReg\210607_shhGFP_HuClyntagRFP"
original_path = c_path + r"\original"

In [5]:
if not os.path.exists(original_path):
    print(f"Creating {original_path}")
    os.makedirs(original_path, exist_ok=True)

In [4]:
ref_ch_num = int(input("Enter Reference Channel Number:"))
line_name = input("Enter Line_name:")
tag_name = input("Enter tag name or name of the stain used:")


Enter Reference Channel Number: 1
Enter Line_name: shhGFP
Enter tag name or name of the stain used: HuClyntagRFP


In [6]:
for file in os.listdir(c_path):
    if file.endswith(".czi"):
        print(file)
        name, ext = file.split(".")
        c = AICSImage(os.path.join(c_path, file))

        # Obtain the image data from respective channels
        if ref_ch_num == 0:
            ref_channel_data = c.get_image_data("ZYX", C=0, S=0, T=0)
            sig_channel_data = c.get_image_data("ZYX", C=1, S=0, T=0)
            
        else:
            ref_channel_data = c.get_image_data("ZYX", C=1, S=0, T=0)
            sig_channel_data = c.get_image_data("ZYX", C=0,S=0, T=0)


        RImage = np.stack(ref_channel_data).astype('uint8')
        SImage = np.stack(sig_channel_data).astype('uint8')
        
#         print(len(RImage))
#         print(len(SImage))

        with tiff.TiffWriter(os.path.join(original_path, f"{name}_ref.tif"), imagej=True) as tifw:
                tifw.write(RImage)

        with tiff.TiffWriter(os.path.join(original_path, f"{name}_sig.tif"), imagej=True) as tifw:
                tifw.write(SImage)
        

shhGFP_1_HuClyntagRFP.czi


C:\Users\keshavgubbi\Anaconda3\lib\site-packages\aicsimageio\transforms.py:177: UserWarning: Data has dimension B with depth 1, assuming B=0 is the desired value, if not the case specify B=x where x is an integer, list, tuple, range, or slice.
  warnings.warn(


shhGFP_2_HuClyntagRFP.czi
shhGFP_3_HuClyntagRFP.czi
shhGFP_4_HuClyntagRFP.czi
shhGFP_5_HuClyntagRFP.czi
shhGFP_6_HuClyntagRFP.czi
shhGFP_7_HuClyntagRFP.czi
shhGFP_8_HuClyntagRFP.czi
shhGFP_9_HuClyntagRFP.czi


In [ ]:
# #start = time.time()
# for file in os.listdir(c_path):
#     if file.endswith(".czi"):
#         print(file)
#         name, ext = file.split(".")

#         czi = CziFile(os.path.join(c_path, file))
#         max_channels = range(*czi.dims_shape()[0]["C"])
#         max_slices = range(*czi.dims_shape()[0]["Z"])
        
#         # for ch_num in max_channels:
#         for ch_num in range(len(max_channels)):
#             print('Processing ch_num:', ch_num, '...........')
#             ref_image_list = sig_image_list = []
#             for z_plane in max_slices:
#                 imgarray, shp = czi.read_image(B=0, S=0, C=ch_num, T=0, Z=z_plane)
#                 print(f"Working with slice {z_plane}....")
#                 if ch_num == ref_ch_num:
#                     ref_image_list.append(np.squeeze(imgarray))
#                 elif ch_num != ref_ch_num:
#                     sig_image_list.append(np.squeeze(imgarray))  
#                 else:
#                     print('Exceeded max channels!')
#         RImage = np.stack(ref_image_list).astype('uint8')
#         SImage = np.stack(sig_image_list).astype('uint8')  
        
#         print(len(RImage))
#         print(len(SImage))
#         print(f"Adding slice {z_plane} to {name}_ref.tif")
#         with tiff.TiffWriter(os.path.join(original_path, f"{name}_ref.tif"), imagej=True) as tifw:
#                 tifw.write(RImage)
#         print(f"Adding slice {z_plane} to {name}_sig.tif")
#         with tiff.TiffWriter(os.path.join(original_path, f"{name}_sig.tif"), imagej=True) as tifw:
#                 tifw.write(SImage)

## Image Processing the tif files

In [7]:
line_path = original_path
processed_path = line_path + "/processed/"
processed_for_average_path = line_path + "/processed_for_average/"

In [8]:
def image_details(f):
    print("********Image Details**************")
    img = io.imread(f)
    print(type(img))
    print("Dtype:", img.dtype)
    print("Shape:", img.shape)
    return img


def convert_to_8bit(f):
    print("********Checking Image Type********")
    # img = io.imread(f)
    dtype = f.dtype
    print(dtype)
    if dtype != "uint8":
        f.astype("uint8", copy=False)
        print(dtype)
    return f.astype("uint8")


def read_voxel_size(f):
    # print("********Reading Voxel Size********")
    im = sitk.ReadImage(f)
    width, height = im.GetSpacing()
    # print('Current voxel size:', width, height)
    return width, height


def ce(f):
    # i = img_as_float(io.imread(f)).astype(np.float64)
    logarithmic_corrected = exposure.adjust_log(f, 3)
    return logarithmic_corrected


def _rotate(src, angle):
    # angle in degrees
    rotated_matrix = ndimage.rotate(src, angle=angle, reshape=False)
    return rotated_matrix


def tiff_unstackAndrestackForRotate(f):
    """
    :param f: tiff file
    :return: rotated_image_stack
    #1. Iterate through each file as a tiff file.
    #2. split into individual pages //Unstacking
    #3. rotate each page and save the rotated_page into a new list
    #4. restack each array from the list
    """
    with tiff.TiffFile(f, mode="r+b") as tif:
        print(f" Processing {tif} for rotation...")
        for page in tif.pages:
            rotated_page = _rotate(page.asarray(), theta)
            rotated_page_list.append(rotated_page)
            rotated_image_stack = np.stack(rotated_page_list)
    return rotated_image_stack.astype("uint8")

#Contrast Enhancement Function
alpha = 10.0 # Contrast control (1.0-3.0) but 5 is required for my purposes here
beta = 5 # Brightness control (0-100). Not to be added beyond 5, to not hamper the signal with unnecesary salt and peper noise.
def tiff_UnstackAndrestackForContrastEnhancement(f):
    """
    :param f: tiff file
    :return: contrast_enhanced_image_stack
    #1. Iterate through each file as a tiff file.
    #2. Split into individual pages //Unstacking and convert each page into an 2D array
    #3. Enhance Contrast for each page and save the contrast_enhanced_page 
    #3a. then pass through bilat filter to remove any residual salt and pepper noise
    #4. Add each page into contrast_enhanced_list and stack the list
    #5. Return the stack.
    """
    contrast_enhanced_list = []
    with tiff.TiffFile(f, mode="r+b") as tif:
        print(f" Processing {tif} for Enhancing Contrast...")
        for page in tif.pages:
            contrast_enhanced_page = cv.convertScaleAbs(page.asarray(), alpha=alpha, beta=beta)
            #Adding a Bilat filter to reduce salt and pepper noise
            contrast_enhanced_page= cv.bilateralFilter(contrast_enhanced_page,9,75,75)
            contrast_enhanced_list.append(contrast_enhanced_page)
            contrast_enhanced_stack = np.stack(contrast_enhanced_list)
    return contrast_enhanced_stack.astype("uint8")

def split_and_rename(f):
    filename, exte = f.split(".")
    _first, _last = filename.split("_", 1)
    fishnum, tag = _last.split("_", 1)
    # print(_first, fishnum, ext)
    return _first, fishnum

In [9]:
for item in os.listdir(line_path):
    if item.endswith(".tif"):

        # ****Contrast Enhancement, 8bit conversion, fixing voxel Depth******#
        g = tiff.imread(os.path.join(line_path, item))
        print(f"The file {item} has dimensions : {g.shape} and is of type: {g.dtype} ")

        CE_image = tiff_UnstackAndrestackForContrastEnhancement(os.path.join(line_path, item))
        
        print(f"Creating file {item} ...")
        with tiff.TiffWriter(os.path.join(line_path, item), imagej=True) as tifw:
            tifw.write(CE_image.astype("uint8"),metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"})

#         #***********Rotation*********************************#
        print(f'Image stack to be rotated: {item}')
        theta = float(input('Enter the angle by which image to be rotated:'))
        rotated_page_list = []
        rotated_image = tiff_unstackAndrestackForRotate(os.path.join(line_path, item))
        print(f'Creating Rotated Image: rotated_{item}')
        with tiff.TiffWriter(os.path.join(line_path, f"{item}"), imagej=True) as tifw:
            tifw.write(rotated_image.astype('uint8'), metadata={'spacing': 1.0, 'unit': 'um', 'axes': 'ZYX'})
        print('*********************************************************')

The file shhGFP_1_HuClyntagRFP_ref.tif has dimensions : (331, 1946, 1024) and is of type: uint8 
 Processing TiffFile 'shhGFP_1_HuClyntagRFP_ref.tif'  629.07 MiB  imagej  331 Pages for Enhancing Contrast...
Creating file shhGFP_1_HuClyntagRFP_ref.tif ...
Image stack to be rotated: shhGFP_1_HuClyntagRFP_ref.tif


Enter the angle by which image to be rotated: 1


 Processing TiffFile 'shhGFP_1_HuClyntagRFP_ref.tif'  629.07 MiB  imagej  331 Pages for rotation...
Creating Rotated Image: rotated_shhGFP_1_HuClyntagRFP_ref.tif
*********************************************************
The file shhGFP_1_HuClyntagRFP_sig.tif has dimensions : (331, 1946, 1024) and is of type: uint8 
 Processing TiffFile 'shhGFP_1_HuClyntagRFP_sig.tif'  629.07 MiB  imagej  331 Pages for Enhancing Contrast...
Creating file shhGFP_1_HuClyntagRFP_sig.tif ...
Image stack to be rotated: shhGFP_1_HuClyntagRFP_sig.tif


Enter the angle by which image to be rotated: 1


 Processing TiffFile 'shhGFP_1_HuClyntagRFP_sig.tif'  629.07 MiB  imagej  331 Pages for rotation...
Creating Rotated Image: rotated_shhGFP_1_HuClyntagRFP_sig.tif
*********************************************************
The file shhGFP_2_HuClyntagRFP_ref.tif has dimensions : (377, 1946, 1024) and is of type: uint8 
 Processing TiffFile 'shhGFP_2_HuClyntagRFP_ref.tif'  716.49 MiB  imagej  377 Pages for Enhancing Contrast...
Creating file shhGFP_2_HuClyntagRFP_ref.tif ...
Image stack to be rotated: shhGFP_2_HuClyntagRFP_ref.tif


...

 Processing TiffFile 'shhGFP_8_HuClyntagRFP_sig.tif'  653.77 MiB  imagej  344 Pages for rotation...
Creating Rotated Image: rotated_shhGFP_8_HuClyntagRFP_sig.tif
*********************************************************
The file shhGFP_9_HuClyntagRFP_ref.tif has dimensions : (315, 1941, 1025) and is of type: uint8 
 Processing TiffFile 'shhGFP_9_HuClyntagRFP_ref.tif'  597.70 MiB  imagej  315 Pages for Enhancing Contrast...
Creating file shhGFP_9_HuClyntagRFP_ref.tif ...
Image stack to be rotated: shhGFP_9_HuClyntagRFP_ref.tif


Enter the angle by which image to be rotated: 2


 Processing TiffFile 'shhGFP_9_HuClyntagRFP_ref.tif'  597.70 MiB  imagej  315 Pages for rotation...
Creating Rotated Image: rotated_shhGFP_9_HuClyntagRFP_ref.tif
*********************************************************
The file shhGFP_9_HuClyntagRFP_sig.tif has dimensions : (315, 1941, 1025) and is of type: uint8 
 Processing TiffFile 'shhGFP_9_HuClyntagRFP_sig.tif'  597.70 MiB  imagej  315 Pages for Enhancing Contrast...
Creating file shhGFP_9_HuClyntagRFP_sig.tif ...
Image stack to be rotated: shhGFP_9_HuClyntagRFP_sig.tif


Enter the angle by which image to be rotated: 2


 Processing TiffFile 'shhGFP_9_HuClyntagRFP_sig.tif'  597.70 MiB  imagej  315 Pages for rotation...
Creating Rotated Image: rotated_shhGFP_9_HuClyntagRFP_sig.tif
*********************************************************


In [11]:
# **********Final Saving of Images to respective folders********
print("Final Saving of Images to respective folders!")
if not os.path.exists(processed_path):
    print(f"Creating {processed_path}")
    os.makedirs(processed_path, exist_ok=True)

if not os.path.exists(processed_for_average_path):
    print(f"Creating {processed_for_average_path}")
    os.makedirs(processed_for_average_path, exist_ok=True)

Final Saving of Images to respective folders!
Creating C:\Users\keshavgubbi\Desktop\LinesReg\210607_shhGFP_HuClyntagRFP\original/processed/
Creating C:\Users\keshavgubbi\Desktop\LinesReg\210607_shhGFP_HuClyntagRFP\original/processed_for_average/


In [12]:
for item in os.listdir(line_path):
    if item.endswith(".tif") and re.search("ref", str(item)):
        print(f"Image stack to be saved: {item}")
        # Read the data back from file
        readdata = tiff.imread(os.path.join(line_path, item))
        name, fn = split_and_rename(item)

        print(f"Creating {name}_{fn}_{tag_name}.nrrd")
        nrrd.write(os.path.join(processed_path, f"{name}_{fn}_{tag_name}.nrrd"),readdata,index_order="C",)

        print(f"Creating {name}_{fn}_{tag_name}.tif")
        with tiff.TiffWriter(os.path.join(processed_for_average_path, f"{name}_{fn}_{tag_name}.tif"),imagej=True,) as tifw:
            tifw.write(readdata.astype("uint8"),metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},)

Image stack to be saved: shhGFP_1_HuClyntagRFP_ref.tif
Creating shhGFP_1_HuClyntagRFP.nrrd
Creating shhGFP_1_HuClyntagRFP.tif
Image stack to be saved: shhGFP_2_HuClyntagRFP_ref.tif
Creating shhGFP_2_HuClyntagRFP.nrrd
Creating shhGFP_2_HuClyntagRFP.tif
Image stack to be saved: shhGFP_3_HuClyntagRFP_ref.tif
Creating shhGFP_3_HuClyntagRFP.nrrd
Creating shhGFP_3_HuClyntagRFP.tif
Image stack to be saved: shhGFP_4_HuClyntagRFP_ref.tif
Creating shhGFP_4_HuClyntagRFP.nrrd
Creating shhGFP_4_HuClyntagRFP.tif
Image stack to be saved: shhGFP_5_HuClyntagRFP_ref.tif
Creating shhGFP_5_HuClyntagRFP.nrrd
Creating shhGFP_5_HuClyntagRFP.tif
Image stack to be saved: shhGFP_6_HuClyntagRFP_ref.tif
Creating shhGFP_6_HuClyntagRFP.nrrd
Creating shhGFP_6_HuClyntagRFP.tif
Image stack to be saved: shhGFP_7_HuClyntagRFP_ref.tif
Creating shhGFP_7_HuClyntagRFP.nrrd
Creating shhGFP_7_HuClyntagRFP.tif
Image stack to be saved: shhGFP_8_HuClyntagRFP_ref.tif
Creating shhGFP_8_HuClyntagRFP.nrrd
Creating shhGFP_8_HuClyntag

In [13]:
for item in os.listdir(line_path):
    if item.endswith(".tif") and re.search("sig", str(item)):
        # Read the data back from file
        readdata = tiff.imread(os.path.join(line_path, item))
        name, fn = split_and_rename(item)

        print(f"Creating {name}_{fn}_GFP.nrrd")
        nrrd.write(os.path.join(processed_path, f"{name}_{fn}_GFP.nrrd"),readdata,index_order="C",)

        print(f"Creating {name}_{fn}_GFP.tif")
        with tiff.TiffWriter(os.path.join(processed_for_average_path, f"{name}_{fn}_GFP.tif"),imagej=True,) as tifw:
            tifw.write(readdata.astype("uint8"),metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},)


Creating shhGFP_1_GFP.nrrd
Creating shhGFP_1_GFP.tif
Creating shhGFP_2_GFP.nrrd
Creating shhGFP_2_GFP.tif
Creating shhGFP_3_GFP.nrrd
Creating shhGFP_3_GFP.tif
Creating shhGFP_4_GFP.nrrd
Creating shhGFP_4_GFP.tif
Creating shhGFP_5_GFP.nrrd
Creating shhGFP_5_GFP.tif
Creating shhGFP_6_GFP.nrrd
Creating shhGFP_6_GFP.tif
Creating shhGFP_7_GFP.nrrd
Creating shhGFP_7_GFP.tif
Creating shhGFP_8_GFP.nrrd
Creating shhGFP_8_GFP.tif
Creating shhGFP_9_GFP.nrrd
Creating shhGFP_9_GFP.tif
